In [39]:
from datetime import datetime
import json
import re
from os import path, environ
from time import sleep
import os

import requests
from geopy.geocoders import GoogleV3
from bs4 import BeautifulSoup
from dateutil.parser import parse

##### Above are the necessary package imports to run the code. The geocoders import is not used in this script currently though. 

In [40]:
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10.5; en-US; rv:1.9.1b3) Gecko/20090305 Firefox/3.1b3 GTB5"
}


In [41]:
# Change the path between the '' to where the accidental_shootings.json file is in your machine for this code to run
ACCIDENTAL_SHOOTINGS_JSON = 'accidental_shootings.json'

# Change the path between the '' to where the accidental_shootings_compact.json file is in your machine for this code to run
ACCIDENTAL_SHOOTINGS_COMPACT_JSON = 'accidental_shootings_compact.json'


##### The code section above is declaring the path for the accidental shootings json file. To run this code on your machine you will need to change the path to where you have saved the accidental_shootings.json file in your machine. This is temporary till there is a more elegant manner to make this step generalizable. 

In [42]:
start_year = 2014
current_year = datetime.now().year
with open(ACCIDENTAL_SHOOTINGS_JSON) as json_file:
    accidental_shooting_data = json.load(json_file)

keyed_accidental_shooting_data = {}
for shooting in accidental_shooting_data:
    keyed_accidental_shooting_data[shooting["id"]] = shooting
    shooting["date"] = parse(shooting["date"])

print(f"Loading existing accidental shooting data with {len(accidental_shooting_data)} incidents")


Loading existing accidental shooting data with 5506 incidents


##### The section above sets the year to start scraping from in our case it is set to 2014. It also opens the accidental_shootings.json file and checks to see how many incidents are there in the file currently.

### Jason edit this part

I want to change the directory of the data scraping. Since we have a limitation from this category, I want to utilize query from GVA system.

In [15]:
from selenium import webdriver      # imports
from selenium.webdriver.support.ui import Select

In [61]:
# Get Chrome driver from https://chromedriver.storage.googleapis.com/



driver = webdriver.Chrome('/Users/jasonjklim/Desktop/EFGS/GVA-master/Accidental Shootings/chromedriver')

# open some page using get method       - url -- > parameters

driver.get('https://www.gunviolencearchive.org/query')

# search tag using id - search add a filter and click

add_rule_bar = driver.find_element_by_class_name('filter-dropdown-trigger')
add_rule_bar.click()

# Choose Date

driver.find_element_by_link_text('Date').click()

# 5 seconds for page loading
sleep(5)

# pull out calender
from_date = driver.find_element_by_css_selector("input[id$='filter-field-date-from']")
from_date.click()
# month select
month = Select(driver.find_element_by_class_name('monthselect'))
sleep(2)
month.select_by_index(0)
sleep(2)

# year select
year = Select(driver.find_element_by_class_name('yearselect'))
sleep(2)
year.select_by_visible_text("2014")

# Date select

all_options = driver.find_elements_by_class_name("available")
for option in all_options:
    print("Value is: %s" % option.get_attribute("value"))
    option.click()





Value is: None


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=80.0.3987.149)


## For next update, currently, first year is 1970, so I will get right year and date for date range and selection of category.


In [16]:
def scrape_results(year, page, headers):
    url = [f"https://www.gunviolencearchive.org/accidental-deaths?page={page}&year={year}", f"https://www.gunviolencearchive.org/accidental-injuries?page={page}&year={year}" ]
    for urls in url: 
        res = requests.get(urls, headers=headers)
        soup = BeautifulSoup(res.text, features="lxml")
    table = soup.find("table")
    headers = [
        re.sub(r"[^a-z]", "", th.get_text().lower()) for th in table.find_all("th")
    ]
    rows = table.find("tbody").find_all("tr")

    results = []
    for row in rows:
        cols = row.find_all("td")
        links = row.select(".links a")
        incident_id = int(re.sub(r"[^\d]", "", links[0].get("href")))
        if not incident_id > 0:
            raise RuntimeError(f"Invalid incident ID {incident_id} at {url}")
        results.append(
            {
                "id": int(re.sub(r"[^\d]", "", links[0].get("href"))),
                "date": parse(cols[headers.index("incidentdate")].get_text()),
                "state": cols[headers.index("state")].get_text(),
                "city": cols[headers.index("cityorcounty")].get_text(),
                "address": cols[headers.index("address")].get_text(),
                "killed": int(cols[headers.index("killed")].get_text()),
                "injured": int(cols[headers.index("injured")].get_text()),
                "source": links[1].get("href") if len(links) > 1 else None,
            }
        )
    return results


##### The section above is creating the web scraper. It is using www.gunviolencearchive.org/accidental-deaths and https://www.gunviolencearchive.org/accidental-injuries as URLs to start from. The URLs in the code block above look slightly different because they use SQL statements to query the GVA website to go through the page and years. 

In [17]:
def merge_shooting_data(scraped_data):
    total_new = 0
    for shooting in scraped_data:
        if shooting["id"] not in keyed_accidental_shooting_data:
            print(f'New shooting found: {shooting["id"]}, {shooting["date"]}')
            keyed_accidental_shooting_data[shooting["id"]] = {}
            total_new += 1
        keyed_accidental_shooting_data[shooting["id"]].update(shooting)
    return total_new




##### The code above is for a function that merges the new results found from scraping the GVA website to update the accidental_shootings.json file. It checks if the 'id' is already present in the file and if it is not the file is updated with the entry.  

In [18]:
for year in range(start_year, current_year + 1):
    print(f"scraping year {year}")
    page = 0
    done_importing = False
    while not done_importing:
        results = scrape_results(year, page, headers)
        new_results_count = merge_shooting_data(results)
        done_importing = new_results_count == 0
        page += 1
        sleep(1)
    print(f"Done importing {year}")

# print("Geocoding missing data")
# for shooting in keyed_mass_shooting_data.values():
#     if not shooting.get("lat") or not shooting.get("lng"):
#         loc = geocode(shooting["address"], shooting["city"], shooting["state"])
#         shooting["lat"] = loc.latitude
#         shooting["lng"] = loc.longitude
#         print(f"geocoded {loc.address}: {loc.latitude}, {loc.longitude}")



scraping year 2014
New shooting found: 272057, 2014-12-31 00:00:00
New shooting found: 272746, 2014-12-31 00:00:00
New shooting found: 272013, 2014-12-31 00:00:00
New shooting found: 275839, 2014-12-31 00:00:00
New shooting found: 271119, 2014-12-30 00:00:00
New shooting found: 273058, 2014-12-30 00:00:00
New shooting found: 271525, 2014-12-30 00:00:00
New shooting found: 284008, 2014-12-29 00:00:00
New shooting found: 270252, 2014-12-28 00:00:00
New shooting found: 270757, 2014-12-27 00:00:00
New shooting found: 269633, 2014-12-27 00:00:00
New shooting found: 281784, 2014-12-27 00:00:00
New shooting found: 269610, 2014-12-27 00:00:00
New shooting found: 284079, 2014-12-27 00:00:00
New shooting found: 284074, 2014-12-27 00:00:00
New shooting found: 269718, 2014-12-27 00:00:00
New shooting found: 270911, 2014-12-27 00:00:00
New shooting found: 269304, 2014-12-26 00:00:00
New shooting found: 270677, 2014-12-26 00:00:00
New shooting found: 270617, 2014-12-26 00:00:00
New shooting found: 2

New shooting found: 221284, 2014-11-08 00:00:00
New shooting found: 217613, 2014-11-07 00:00:00
New shooting found: 217617, 2014-11-07 00:00:00
New shooting found: 216681, 2014-11-06 00:00:00
New shooting found: 217628, 2014-11-06 00:00:00
New shooting found: 216963, 2014-11-05 00:00:00
New shooting found: 219020, 2014-11-05 00:00:00
New shooting found: 216142, 2014-11-04 00:00:00
New shooting found: 215430, 2014-11-04 00:00:00
New shooting found: 215446, 2014-11-04 00:00:00
New shooting found: 219058, 2014-11-03 00:00:00
New shooting found: 215366, 2014-11-03 00:00:00
New shooting found: 214889, 2014-11-03 00:00:00
New shooting found: 215002, 2014-11-03 00:00:00
New shooting found: 214629, 2014-11-02 00:00:00
New shooting found: 220012, 2014-11-02 00:00:00
New shooting found: 214749, 2014-11-02 00:00:00
New shooting found: 215087, 2014-11-01 00:00:00
New shooting found: 1557623, 2014-11-01 00:00:00
New shooting found: 222610, 2014-11-01 00:00:00
New shooting found: 214355, 2014-11-01 

New shooting found: 191194, 2014-09-12 00:00:00
New shooting found: 191196, 2014-09-12 00:00:00
New shooting found: 189902, 2014-09-12 00:00:00
New shooting found: 189005, 2014-09-11 00:00:00
New shooting found: 196020, 2014-09-11 00:00:00
New shooting found: 189664, 2014-09-11 00:00:00
New shooting found: 189177, 2014-09-10 00:00:00
New shooting found: 189242, 2014-09-10 00:00:00
New shooting found: 194568, 2014-09-10 00:00:00
New shooting found: 189334, 2014-09-10 00:00:00
New shooting found: 187596, 2014-09-08 00:00:00
New shooting found: 187916, 2014-09-08 00:00:00
New shooting found: 187911, 2014-09-08 00:00:00
New shooting found: 189846, 2014-09-08 00:00:00
New shooting found: 187267, 2014-09-07 00:00:00
New shooting found: 188051, 2014-09-07 00:00:00
New shooting found: 186172, 2014-09-05 00:00:00
New shooting found: 186854, 2014-09-05 00:00:00
New shooting found: 187598, 2014-09-05 00:00:00
New shooting found: 186317, 2014-09-05 00:00:00
New shooting found: 186443, 2014-09-05 0

New shooting found: 465298, 2015-12-13 00:00:00
New shooting found: 465277, 2015-12-13 00:00:00
New shooting found: 465332, 2015-12-13 00:00:00
New shooting found: 465293, 2015-12-13 00:00:00
New shooting found: 464077, 2015-12-12 00:00:00
New shooting found: 465282, 2015-12-12 00:00:00
New shooting found: 464648, 2015-12-12 00:00:00
New shooting found: 464183, 2015-12-12 00:00:00
New shooting found: 465820, 2015-12-12 00:00:00
New shooting found: 466939, 2015-12-11 00:00:00
New shooting found: 464099, 2015-12-11 00:00:00
New shooting found: 1433162, 2015-12-11 00:00:00
New shooting found: 474425, 2015-12-11 00:00:00
New shooting found: 463672, 2015-12-10 00:00:00
New shooting found: 463261, 2015-12-10 00:00:00
New shooting found: 463370, 2015-12-10 00:00:00
New shooting found: 461582, 2015-12-08 00:00:00
New shooting found: 462575, 2015-12-08 00:00:00
New shooting found: 462876, 2015-12-08 00:00:00
New shooting found: 474480, 2015-12-08 00:00:00
New shooting found: 462365, 2015-12-08 

New shooting found: 440146, 2015-10-31 00:00:00
New shooting found: 440986, 2015-10-31 00:00:00
New shooting found: 443388, 2015-10-31 00:00:00
New shooting found: 439772, 2015-10-31 00:00:00
New shooting found: 441706, 2015-10-31 00:00:00
New shooting found: 439854, 2015-10-31 00:00:00
New shooting found: 440265, 2015-10-31 00:00:00
New shooting found: 440974, 2015-10-31 00:00:00
New shooting found: 444968, 2015-10-31 00:00:00
New shooting found: 439322, 2015-10-30 00:00:00
New shooting found: 439873, 2015-10-30 00:00:00
New shooting found: 439235, 2015-10-30 00:00:00
New shooting found: 440117, 2015-10-30 00:00:00
New shooting found: 441303, 2015-10-29 00:00:00
New shooting found: 439412, 2015-10-29 00:00:00
New shooting found: 439018, 2015-10-29 00:00:00
New shooting found: 439251, 2015-10-29 00:00:00
New shooting found: 438485, 2015-10-28 00:00:00
New shooting found: 438251, 2015-10-28 00:00:00
New shooting found: 438726, 2015-10-28 00:00:00
New shooting found: 438121, 2015-10-28 0

...

New shooting found: 1580342, 2019-12-26 00:00:00
New shooting found: 1580494, 2019-12-26 00:00:00
New shooting found: 1579448, 2019-12-25 00:00:00
New shooting found: 1582619, 2019-12-25 00:00:00
New shooting found: 1580276, 2019-12-25 00:00:00
New shooting found: 1579726, 2019-12-25 00:00:00
New shooting found: 1579962, 2019-12-25 00:00:00
New shooting found: 1579753, 2019-12-23 00:00:00
New shooting found: 1578832, 2019-12-23 00:00:00
New shooting found: 1577878, 2019-12-22 00:00:00
New shooting found: 1578388, 2019-12-22 00:00:00
New shooting found: 1578404, 2019-12-22 00:00:00
New shooting found: 1578016, 2019-12-22 00:00:00
New shooting found: 1578357, 2019-12-22 00:00:00
New shooting found: 1578055, 2019-12-22 00:00:00
New shooting found: 1579183, 2019-12-21 00:00:00
New shooting found: 1577347, 2019-12-21 00:00:00
New shooting found: 1577165, 2019-12-21 00:00:00
New shooting found: 1576598, 2019-12-20 00:00:00
New shooting found: 1575947, 2019-12-19 00:00:00
New shooting found: 

New shooting found: 1549242, 2019-11-13 00:00:00
New shooting found: 1548953, 2019-11-13 00:00:00
New shooting found: 1547198, 2019-11-12 00:00:00
New shooting found: 1547789, 2019-11-12 00:00:00
New shooting found: 1546923, 2019-11-11 00:00:00
New shooting found: 1553971, 2019-11-11 00:00:00
New shooting found: 1547117, 2019-11-11 00:00:00
New shooting found: 1547094, 2019-11-10 00:00:00
New shooting found: 1546051, 2019-11-10 00:00:00
New shooting found: 1546044, 2019-11-10 00:00:00
New shooting found: 1545663, 2019-11-09 00:00:00
New shooting found: 1545613, 2019-11-09 00:00:00
New shooting found: 1547668, 2019-11-09 00:00:00
New shooting found: 1545621, 2019-11-09 00:00:00
New shooting found: 1546257, 2019-11-09 00:00:00
New shooting found: 1547793, 2019-11-09 00:00:00
New shooting found: 1545693, 2019-11-09 00:00:00
New shooting found: 1544824, 2019-11-08 00:00:00
New shooting found: 1545769, 2019-11-08 00:00:00
New shooting found: 1548609, 2019-11-08 00:00:00
New shooting found: 

New shooting found: 1513593, 2019-09-26 00:00:00
New shooting found: 1518491, 2019-09-26 00:00:00
New shooting found: 1513731, 2019-09-25 00:00:00
New shooting found: 1511861, 2019-09-24 00:00:00
New shooting found: 1523142, 2019-09-23 00:00:00
New shooting found: 1512670, 2019-09-23 00:00:00
New shooting found: 1599654, 2019-09-23 00:00:00
New shooting found: 1510614, 2019-09-23 00:00:00
New shooting found: 1512046, 2019-09-23 00:00:00
New shooting found: 1510020, 2019-09-22 00:00:00
New shooting found: 1510857, 2019-09-22 00:00:00
New shooting found: 1509882, 2019-09-22 00:00:00
New shooting found: 1510131, 2019-09-22 00:00:00
New shooting found: 1510016, 2019-09-22 00:00:00
New shooting found: 1510433, 2019-09-22 00:00:00
New shooting found: 1511211, 2019-09-22 00:00:00
New shooting found: 1510990, 2019-09-21 00:00:00
New shooting found: 1508328, 2019-09-20 00:00:00
New shooting found: 1508681, 2019-09-20 00:00:00
New shooting found: 1510123, 2019-09-20 00:00:00
New shooting found: 

New shooting found: 1631278, 2020-02-18 00:00:00
New shooting found: 1624441, 2020-02-18 00:00:00
New shooting found: 1624304, 2020-02-18 00:00:00
New shooting found: 1624943, 2020-02-18 00:00:00
New shooting found: 1624265, 2020-02-18 00:00:00
New shooting found: 1623862, 2020-02-17 00:00:00
New shooting found: 1630195, 2020-02-17 00:00:00
New shooting found: 1625298, 2020-02-17 00:00:00
New shooting found: 1623784, 2020-02-17 00:00:00
New shooting found: 1622932, 2020-02-17 00:00:00
New shooting found: 1624268, 2020-02-17 00:00:00
New shooting found: 1623933, 2020-02-16 00:00:00
New shooting found: 1624596, 2020-02-16 00:00:00
New shooting found: 1622589, 2020-02-16 00:00:00
New shooting found: 1622566, 2020-02-16 00:00:00
New shooting found: 1622693, 2020-02-16 00:00:00
New shooting found: 1622789, 2020-02-15 00:00:00
New shooting found: 1622312, 2020-02-15 00:00:00
New shooting found: 1623104, 2020-02-15 00:00:00
New shooting found: 1622267, 2020-02-15 00:00:00
New shooting found: 

New shooting found: 1589609, 2020-01-06 00:00:00
New shooting found: 1589915, 2020-01-06 00:00:00
New shooting found: 1589932, 2020-01-06 00:00:00
New shooting found: 1588919, 2020-01-05 00:00:00
New shooting found: 1589251, 2020-01-05 00:00:00
New shooting found: 1588560, 2020-01-04 00:00:00
New shooting found: 1591414, 2020-01-04 00:00:00
New shooting found: 1588399, 2020-01-04 00:00:00
New shooting found: 1589597, 2020-01-04 00:00:00
New shooting found: 1588319, 2020-01-04 00:00:00
New shooting found: 1588978, 2020-01-04 00:00:00
New shooting found: 1589230, 2020-01-03 00:00:00
New shooting found: 1591399, 2020-01-03 00:00:00
New shooting found: 1588903, 2020-01-03 00:00:00
New shooting found: 1587916, 2020-01-03 00:00:00
New shooting found: 1588106, 2020-01-03 00:00:00
New shooting found: 1588678, 2020-01-02 00:00:00
New shooting found: 1587401, 2020-01-02 00:00:00
New shooting found: 1588324, 2020-01-02 00:00:00
New shooting found: 1587560, 2020-01-02 00:00:00
New shooting found: 

In [19]:
print("Sorting results")
updated_shooting_data = sorted(
    keyed_accidental_shooting_data.values(), key=lambda row: row['date']
)

updated_shootings_compact = []
for shooting in updated_shooting_data:
    shooting["date"] = shooting["date"].strftime("%Y-%m-%d")
    updated_shootings_compact.append(
        [
            shooting["id"],
            shooting["date"],
            shooting["killed"],
            shooting["injured"],
           
        ]
    )


print("Writing results")
with open(ACCIDENTAL_SHOOTINGS_JSON, "w") as outfile:
    json.dump(updated_shooting_data, outfile, indent=2)

with open(ACCIDENTAL_SHOOTINGS_COMPACT_JSON, "w") as outfile:
    json.dump(updated_shootings_compact, outfile, separators=(",", ":"))

print(f"Done! {len(updated_shooting_data)} shootings found")

Sorting results
Writing results
Done! 5506 shootings found


##### In the two code blocks above the scrape_results and merge_shooting_data functions are called to iterate over the years from 2014-the current date and the pages that show up for each year.  Then finally the data is sorted based on date and the final result is written to the accidental_shootings.json file.